In [1]:
# import libraries
from jetbot import Robot, Camera, bgr8_to_jpeg
from IPython.display import display
from uuid import uuid1
import ipywidgets.widgets as widgets
import traitlets, time, os
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np
import time

In [2]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)
model.load_state_dict(torch.load('model_stop_02.pth'))
device = torch.device('cuda')
model = model.to(device)

In [3]:
robot = Robot()

In [4]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [5]:
camera = Camera.instance(width=224, height=224, fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

In [7]:
def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    #if prob_blocked < 0.5:
        #robot.forward(0.1)
        #print("free: ", prob_blocked)
    #else:
        #robot.left(0.2)
        #print("blocked: ", prob_blocked)
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

In [8]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
start_bot_button = widgets.Button(description='start bot', button_style='success', layout=button_layout)
stop_bot_button = widgets.Button(description='stop bot', button_style='danger', layout=button_layout)

# define buttons and actions
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.2)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.2)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.1)
    time.sleep(0.4)
    robot.stop()

def step_right(change):
    robot.right(0.1)
    time.sleep(0.4)
    robot.stop()

def start_bot(change):   
    camera.observe(update, names='value')
    
def stop_bot(change):    
    camera.unobserve(update, names='value')
    robot.stop()

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
start_bot_button.on_click(start_bot)
stop_bot_button.on_click(stop_bot)

# display buttons
top_box = widgets.HBox([start_bot_button, forward_button, start_bot_button], layout=widgets.Layout(align_self='center'))
middle_box = widgets.HBox([left_button, backward_button, right_button], layout=widgets.Layout(align_self='center'))
bottom_box = widgets.HBox([stop_bot_button, stop_button, stop_bot_button], layout=widgets.Layout(align_self='center')) 

In [9]:
display(widgets.HBox([image, blocked_slider]))

In [11]:
# display gui
display(top_box)
display(middle_box)
display(bottom_box)